In [ ]:
import os
import cv2
import sheet_id.deepmatching_cpu.deepmatching as dm
from tqdm import tqdm_notebook as tqdm
from sheet_id.utils.base_utils import loadSettings, loadScoresDataset, loadCellphoneScoresDataset, \
    generateScoresDB, calculateMRR

from matplotlib.patches import ConnectionPatch
from multiprocessing import Pool, freeze_support

In [ ]:
settings = loadSettings()

In [ ]:
scores_path = loadScoresDataset(settings['DB_PATH'])
cellphone_scores_path = loadCellphoneScoresDataset(settings['CELLPHONE_IMG_PATH'])
db = generateScoresDB(scores_path)

In [ ]:
def imageDistance(img_a, img_b):
    _, img1 = cv2.threshold(img_a,127,255,cv2.THRESH_BINARY)
    _, img2 = cv2.threshold(img_b,127,255,cv2.THRESH_BINARY)
    img1 = cv2.resize(img1, (0,0), fx=0.3, fy=0.3)
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
    matches = dm.deepmatching(img1, img2, '-downscale 2 -v -nt 10' )
    score = len(matches)
    distance = -score
    return distance

In [ ]:
def search(query_img, db):
    """
    Search the query from the db and return the list of predictions ranked by the score in
    the form of 
        (ref_name, distance)
    """
    scoreRanking = []
    for ref in tqdm(db):
        ref_img = db[ref]
        dist = imageDistance(query_img, ref_img)
        scoreRanking.append((ref, dist))
    return sorted(scoreRanking, key=lambda x: x[1]) # Sort based on distance in increasing order 

In [ ]:
def findRank(predictions, ref):
    """
    Find the rank of ref in the list of predictions
    """
    rank = 1
    for (prediction, dist) in predictions:
        if prediction == ref:
            return rank
        rank += 1
    raise

In [ ]:
ranks = []
for i in tqdm(range(len(cellphone_scores_path))):
    img = cv2.imread(cellphone_scores_path[i], 0)
    fileNameNoExt = os.path.splitext(os.path.split(cellphone_scores_path[i])[1])[0]
    prediction_batches = search(img, db)
    predictions = []
    for prediction_batch in prediction_batches:
        predictions.extend(prediction_batch)
    predictions = sorted(predictions, key=lambda x: x[1])
    rank = findRank(predictions, fileNameNoExt)
    ranks.append(rank)
    print("Query {:} [{:}]: {:}".format(i+1, fileNameNoExt, rank))

In [ ]:
MRR = calculateMRR(ranks)
print("MRR: {:}".format(MRR))

# Deepmatching Testing

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(test_img_a, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(test_img_b, cmap='gray')
plt.show()

In [ ]:
_, img1 = cv2.threshold(test_img_a,127,255,cv2.THRESH_BINARY)
_, img2 = cv2.threshold(test_img_b,127,255,cv2.THRESH_BINARY)
img1 = cv2.resize(img1, (0,0), fx=0.3, fy=0.3)
img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(img1, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img2, cmap='gray')
plt.show()

In [ ]:
dm.deepmatching()

In [ ]:
matches = dm.deepmatching(img1, img2, '-downscale 2 -v' )

In [ ]:
fig = plt.figure(figsize=(20,20))

ax1 = fig.add_subplot(1,2,1)
ax1.imshow(img1)
for i in range(len(matches)):
    ax1.plot([matches[i][0]], [matches[i][1]], 'ko')
           
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(img2)
for i in range(len(matches)):
    ax2.plot([matches[i][2]], [matches[i][3]], 'ko')

for i in range(len(matches)):
    con = ConnectionPatch(xyA=(matches[i][2], matches[i][3]), coordsA='data', coordsB='data',
                          xyB=(matches[i][0], matches[i][1]), axesA=ax2, axesB=ax1, color='red')
    ax2.add_artist(con)

In [ ]:
imageDistance(test_img_a, test_img_b)